In [22]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Rea

In [23]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [24]:
!pip install sentence-transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from glob import glob
import re

import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import torch

In [26]:
def load_data(file, sheet_name):
    return pd.read_excel(file, sheet_name=sheet_name)

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
def load_all_data(path, sheet):
    files = glob(path)
    df = pd.DataFrame()
    for file in files:
        df = df.append(load_data(file, sheet))
    return df

news = load_all_data('/content/drive/MyDrive/aida/dataset/임대차3법(54,752건)/*.xlsx', '뉴스')

In [32]:
print(news.shape)
news.head()

(17497, 4)


,작성일,작성자,제목,내용
1,2020/11/17,MBC연예,"'100분토론' 서민 주거 안정 위한 '임대차 3법', 10명 중 6명은 모른다?",오늘 17일 방송되는 MBC 100분 토론 에서는 전세난 해법은 이라는 주제...
2,2020/08/12,오마이뉴스,임대차3법 탓에 10억→14억?... 언론은 어떻게 왜곡하나,보도 검증 기사에 거론된 아파트들 찾아가보니... 원래 14억대 거래... 10...
3,2020/07/29,연합뉴스,"""임대차 3법, 주거안정 기여 기대""…매물잠김 우려도(종합)",전문가들 긍정 평가속 시행 초기 부작용 우려 의사봉 두드리는 윤호중 법사위원장 ...
4,2020/07/29,연합뉴스,"'전세난민' 사라지나…전문가 ""임대차3법, 주거안정에 도움""",전세매물 잠김현상 4년 주기 전셋값 폭등 우려도 의사봉 두드리는 윤호중 법사위원장 ...
5,2020/11/16,한겨레,"[안재승 칼럼] ‘동네북’ 임대차 3법, 더 강력해져야 한다",임대차 3법이 동네북 신세다. 보수 야당과 언론이 임대차 3법이 전세난을 불렀다고 ...


In [39]:
def text_cleaning(x):
    mail_del = re.sub("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z-.]+)", "", str(x))
    meta_del = re.sub("[\r\n\xa0]", "", str(mail_del))
    name_del = re.sub("(\.\s+[ㄱ-ㅎ가-힣]+\s[기]+[자]+)", "", str(meta_del))
    clean_text = re.sub("[^\w\s^.]", " ", name_del)
    
    return clean_text

In [40]:
news.columns = news.iloc[0]
news.drop(0, inplace=True, axis=0)
news = news.dropna(axis=0, how='any')

news['내용'] = news['내용'].map(text_cleaning)
news = news.sort_values(by='작성일')

print(news.shape)
news.head()

(17497, 4)


,작성일,작성자,제목,내용
8938,2020/07/01,MBN,'전세 대란' 본격화…오늘부터 실거주 요건 강화,동영상 뉴스 앵커멘트 6 17 부동산 대책 ...
1566,2020/07/01,국민일보,“임대사업자 등록 인센티브가 과세 형평성 뒤흔든다”[이슈&탐사],정부가 깔아준 다주택 꽃길 ④모순 덩어리 임대사업자 대책 전문가 조언 정부 여당...
8939,2020/07/01,뉴스1,"[씨 마른 전세시장]""전세만 찾아요""…수요 증가에 전셋값 더 뛰나",6월 서울 아파트 전세 거래 비중 78.4 2013년 3월 이후 최고치 세입자 ...
639,2020/07/01,국회,[국회보 2020년 7월호] 경제이야기_‘임대차 3법’의 나비효과,뜨거운 여름이 지나면 아마 임대차 3 법 으로 불리는 새로운 법이 토론의...
274,2020/07/01,데일리안,"거침없는 전셋값 상승, ‘임대차 3법’ 관심 쏠려…“독일, 10년간 57% 상승”",서울 전 지역 전셋값 오름세 귀해진 전세매물에 가격 올라도 거래 임차인 보호 앞...


In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [17]:
# 이미 fine-tuning되어있는 open-source model이 있어 그걸 사용, cost 낭비X
class mySBERT(SentenceTransformer):
    def __init__(self, path, modules=None):
        super().__init__(path, modules)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = self._first_module().tokenizer

    def encode(self, sentences, batch_size=32, show_progress_bar=None, output_value='token_embeddings', convert_to_numpy=True, convert_to_tensor=False, is_pretokenized=False):
        return super().encode(sentences, batch_size, show_progress_bar, output_value, convert_to_numpy, convert_to_tensor, is_pretokenized)

    def tokenize(self, sentences):
        return self.tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    
    def to(self, device):
        self.device = device
        return self

In [65]:
okt = Okt()
model = SentenceTransformer('jhgan/ko-sbert-multitask')

keywords = []
np.random.seed(42)
for i in tqdm(np.random.choice(news.index, 10000)):
    tokenized_doc = okt.pos(news.iloc[i]['내용'], norm=True, stem=True)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

    n_gram_range = (1,2)

    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()

    candidate_embeddings = model.encode(candidates, convert_to_tensor=True)
    doc_embedding = model.encode([tokenized_nouns], convert_to_tensor=True)
    
    keywords.append({news.iloc[i]['작성일']:max_sum_sim(doc_embedding.cpu(), candidate_embeddings.cpu(), candidates, top_n=5, nr_candidates=10)})

100%|██████████| 10000/10000 [58:33<00:00,  2.85it/s]


In [68]:
import json

with open('/content/drive/MyDrive/aida/keywords.json', 'w', encoding='utf-8') as f:
    json.dump(keywords, f, ensure_ascii=False, indent=4)

In [67]:
for word in keywords:
  print(word)

Streaming output truncated to the last 5000 lines.
{'2020/09/07': ['전세 매물', '경우 전세계약', '중심 월세', '월세 과도기', '소형 월세']}
{'2020/11/11': ['때문 전세', '불균형 전세', '전월세 거래', '최근 임대차보호법', '차법 전월세']}
{'2020/07/29': ['법안 단독', '민주당 국회', '계획 국회', '전월세 법안', '법안 후속']}
{'2020/12/03': ['응답 주택', '전월세 시행', '전월세 시장', '내년 집값', '부동산 전국']}
{'2020/07/30': ['통제 강화', '검찰 경찰', '개혁', '권력 기관', '경찰 도입']}
{'2020/10/29': ['대통령 국정', '전세 해결', '임대차 처방', '문재인 대통령', '부동산 안정']}
{'2020/07/30': ['임대차 법안', '소위 상정', '하명 법안', '법안 본회의', '입법 민주당']}
{'2020/07/24': ['지금 전세', '수도권 주택', '셋값 폭등', '집값 폭등', '전세 매물도']}
{'2020/07/07': ['정책 규제', '국회 부동산', '대책 비판', '김현미 국토교통부', '원내대표 김현미']}
{'2020/12/07': ['여야 간사', '국회법 위반', '정무 법안', '법안 심사', '여당 국회법']}
{'2020/08/10': ['부동산 세법', '대한 종부세', '세율 대폭', '세율 주택', '종부세 현행']}
{'2020/08/02': ['통상 조례', '임대료 상한', '임대차보호법 개정안', '지역별 임대료', '지자체 조례']}
{'2020/07/29': ['갱신 임대료', '갱신 청구권', '주택임대차보호법 개정안', '전월세 도입', '임대차 법안']}
{'2020/07/06': ['주택임대차보호법 개정안', '의원 전월세', '전월세 고제', '발의 임대차', '임대차 개정안']}
{'2020/11/23':